In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from numpy import mean
from numpy import std
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from matplotlib import pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/Classification/speech')

In [ ]:
data = pd.read_excel('classification.xlsx', engine='openpyxl')

In [ ]:
X = data.iloc[:,2:12].to_numpy()
y = np.array([0 if x=='no_rad' else 1 for x in data.iloc[:,-1]])

In [ ]:
X = data['Proportion speech child'].to_numpy()
X = X.reshape(-1, 1)

In [ ]:
X

In [ ]:
groups = np.array(data['Subject_ID'])

In [ ]:
X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=1)

In [ ]:
steps = list()
#steps.append(('scaler', StandardScaler()))
steps.append(('model', SVC(kernel='linear')))
pipeline = Pipeline(steps=steps)

In [ ]:
cv = GroupKFold(n_splits=5)

In [ ]:
scores = cross_val_score(pipeline, X_shuffled, y_shuffled, scoring='accuracy', cv=cv, n_jobs=-1, groups=groups_shuffled)

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(model, X_shuffled, y_shuffled, cv=cv, groups = groups_shuffled, n_jobs=-1)
conf_mat = confusion_matrix(y_shuffled, y_pred)

In [ ]:
conf_mat

In [ ]:
print('Accuracy: %.4f (%.3f)' % (mean(scores)*100, std(scores)*100))

In [ ]:
accuracy = []
n_shuffles = 10
coefficients = []
for i in range(n_shuffles):
    X_shuffle, y_shuffle, groups_shuffle = shuffle(X, y, groups, random_state=i)
    scores = cross_val_score(pipeline, X_shuffle, y_shuffle, groups=groups_shuffle, scoring='accuracy', cv=cv )
    print('Accuracy: %.4f (%.3f)' % (mean(scores)*100, std(scores)*100))
    accuracy.append(scores)

In [ ]:
model = SVC(kernel='linear')

In [ ]:
#calculate accuracy and coefficients 
accuracies_combined= []
final_coef =[]
recall_all = []
precision_all=[]
for i in range(10):
    X_shuffle, y_shuffle, groups_shuffle = shuffle(X, y, groups, random_state=i)
    kf = GroupKFold(n_splits=5)
    coefficients = []
    accuracies = []
    recalls = []
    precisions=[]
    for train_index, test_index in kf.split(X_shuffle,y_shuffle, groups_shuffle):

            # scale training and test data based on statistics of only training data
        scaler = StandardScaler()
        X_train_, X_test_ = X_shuffle[train_index], X_shuffle[test_index]
        y_train_, y_test_ = y_shuffle[train_index], y_shuffle[test_index]
        scaler = StandardScaler()
        scaler.fit(X_train_)
            # normalize data
        X_train_, X_test_ = scaler.transform(X_train_), scaler.transform(X_test_)
        model = SVC(kernel='linear', probability=True)
        model.fit(X_train_, y_train_)
        print(model.predict(X_test_))

        accuracy = accuracy_score(y_test_, model.predict(X_test_))
        accuracies.append(accuracy)
        recall = recall_score(y_test_, model.predict(X_test_))
        precision = precision_score(y_test_, model.predict(X_test_))
        recalls.append(recall)
        precisions.append(precision)
        print(f"Accuracy {accuracy:3f}")
        print(model.coef_)
        coefficients.append(model.coef_)
    accuracies_combined.append(mean(accuracies))
    recall_all.append(mean(recalls))
    precision_all.append(mean(precisions))
    final_coef.append(mean(coefficients, axis=1))

In [ ]:
mean( accuracies_combined)

In [ ]:
X

In [ ]:
recall_all, precision_all

In [ ]:
features_names = [data.iloc[:,2:12].columns]

In [ ]:
combined_coefs = mean(abs(final_coef[0]), axis=0)
std_combined = np.std(abs(final_coef[0]), axis=0)

In [ ]:
mean(abs(final_coef[0]), axis=0)

In [ ]:
std_combined, combined_coefs

In [ ]:
combined_features_imp = pd.DataFrame(list(zip(combined_coefs, std_combined)),index=features_names, columns = ['coefficient', 'std'])
combined_features_imp.T.to_csv('combined_coefs.csv', index=False)


In [ ]:
combined_features_imp

In [ ]:

X_shuffle, y_shuffle, groups_shuffle = shuffle(X, y, groups, random_state=i)
kf = GroupKFold(n_splits=5)
predictions = []
labels = []
s_id=[]
for train_index, test_index in kf.split(X_shuffle,y_shuffle, groups_shuffle):

            # scale training and test data based on statistics of only training data
    scaler = StandardScaler()
    X_train_, X_test_ = X_shuffle[train_index], X_shuffle[test_index]
    y_train_, y_test_ = y_shuffle[train_index], y_shuffle[test_index]
    groups_train_, groups_test_ = groups_shuffle[train_index], groups_shuffle[test_index]
    scaler = StandardScaler()
    scaler.fit(X_train_)
            # normalize data
    X_train_, X_test_ = scaler.transform(X_train_), scaler.transform(X_test_)
    model = SVC(kernel='linear', probability=True)
    model.fit(X_train_, y_train_)
    print(model.predict(X_test_))
        

    labels.append(y_test_)
    predictions.append(model.predict(X_test_))
    s_id.append(groups_test_)
    

In [ ]:
y_pred = cross_val_predict(pipeline, X_shuffle, y_shuffle, groups=groups_shuffle, cv=kf, n_jobs=-1)

In [ ]:
k=0
tuple_index_pred = list()
while k<5:
    for x, y,z in zip(s_id[k], predictions[k], labels[k]):
        tuple_index_pred.append((x,y,z))
    k = k+1

In [ ]:
t_list = list()
t_list2 = list()
t_list3=list()
for x in tuple_index_pred:
    t_list.append(x[0])
    t_list2.append(x[1])
    t_list3.append(x[2])

final_df = pd.DataFrame(list(zip(t_list, t_list2, t_list3)), columns = ['s_id', 'prediction', 'label'])
final_df

In [ ]:
final_df = pd.DataFrame(list(zip(labels, predictions, s_id)), columns = ['label', 'prediction', 's_id'])

In [ ]:
final_df

In [ ]:
final_df.loc[final_df['prediction']]

In [ ]:
final_df.loc[final_df['prediction']==1].duplicated().describe() #41 with False 

In [ ]:
final_df.loc[final_df['prediction']==0].duplicated().describe() # 42 with 

In [ ]:
# 1. discard all people where there is a disagree in label 
final_df['discarded'] = final_df[['s_id','prediction']].duplicated(keep=False) # duplicated means True; disagree in label means false

In [ ]:
final_df['discarded']

In [ ]:
final_df[['s_id','prediction']].duplicated(keep=False)

In [ ]:
# keep only those where there is agree  in label ; so duplicated ones 
final_df.loc[final_df['discarded']==True] # so we have 76 where there is agreement in label in the classifier output
# and 41 where there isn't
# so 76 with both recordings in agreement; so actually 38 people out of the 56 that have both recordings
# so the rest of 18 people we leave to clinician

In [ ]:
len(final_df.loc[final_df['discarded']==True])

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(final_df.loc[final_df['discarded']==True].sort_values(by=['s_id']))

In [ ]:
df_disagree = final_df.loc[final_df['discarded']==False]

In [ ]:
df_disagree.sort_values(by=['s_id'])

In [ ]:
df_disagree= df_disagree.drop([113, 52, 89, 43, 31])

In [ ]:
len(df_disagree.loc[df_disagree['label']==0])  # 14 out of 46 
# 7 out of 23 children for whom there was a disagree actually got a no_rad

In [ ]:
final_discarded = final_df.loc[final_df['discarded']==True]

In [ ]:
len(final_discarded) # 38 people with both recordings agreeement

In [ ]:
df = final_discarded.loc[final_discarded['discarded']==True]
accuracy=0
for x,y in zip(final_discarded['label'], final_discarded['prediction']):
        if x==y:
            accuracy +=1
accuracy

In [ ]:
# so 29 are correct out of 56 and 11 are wrong

In [ ]:
accuracy/76

In [ ]:
fp = 0
fn = 0
tp = 0
tn = 0
for x,y in zip(final_discarded['label'], final_discarded['prediction']):
        if x!=y and x==1:
            fp +=1
        if x!=y and x==0:
            fn +=1
        if x==y and x==1:
            tp +=1
        if x==0 and y ==0:
            tn +=1

In [ ]:
recall = tp/(tp+fn)
precision = tp/(tp+fp)
specificity = tn/(tn+fp)
recall, precision, specificity


In [ ]:
df.sort_values(by=['s_id'])

In [ ]:
final_discarded.sort_values(by='s_id')